<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Features-to-add" data-toc-modified-id="Features-to-add-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Features to add</a></span></li></ul></div>

Here I am merging tables onto the main table, transfers.

## Features to add

Adding for sure:

- Prescriptions, which is ready

- Admissions: check ppoint, non-white?, marital status??, medicaid, language?

- Patients: Age and gender

- POE: Many categories (order type), but decent. 


Maybe adding in the future:

- Procedures by ICD code, could be interesting but lots of features

- Lab events: A LOT of features, although you can just add flag (associated with storetime) and priority (associated with charttime of test (frequency encoding?)

- OMR: Blood pressure, weight... Interesting but probably there will be many missing values and will be hard to incorporate into dataframe

- hcpcsevents: A LOT of features

- Microbiology events: A LOT of features (no outcomes clear, although there are comments of test)


<br><br>

Some questions: 

Should I increase the granularity of the prescriptions (go down to level 2, 3 or 4). That would add between 1K and 5K features

Can I leave duration of transfer in datetime or should I categorise it/bin it?
bin

Should I bin age or can be quantitative?
Bin it (maybe check before when it's quantitative)

Is it okay adding POE? Should I stop there or add also the other tables? It would mean lots of features. 

Should I add measures such as blood pressure and weight even if values will be missing in many rows? And again do I have to bin?
Leave it for now and maybe with the PCA it tells you something. Maybe you see that could be good to include

Focus on not adding any extra features but focus on step 1 (do PCA, fit models and decide whether to add more features or not)

Step 1
For small dataset
logistic regression SVM, decision tree and random forest, naive bayes
Group the classes before fitting the model
PCA to see if I can compress the model as it is right now
See if by going up on the number of features you are getting any gain
How many orders for now and that's it

Step 2
For large dataset

For dataset try to make bins if possible, 5-10

Distribution is skewed. But after we have converted (the class) into binary, the problem we have is bias (the model is biased).

In [1]:
import numpy as np
import pandas as pd
import copy
import nachopy
import importlib
import pickle

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats = ['svg'] # Setting plots as vector images

plt.rcParams['figure.figsize'] = (10, 5)
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import matplotlib as mpl
import seaborn as sns

In [2]:
transfers = pd.read_csv('../Capstone data/MIMIC IV/transfers.csv')

In [3]:
transfers.sort_values(by = ['subject_id','intime']).head(10)

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10000032,22595853.0,33258284,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00
1,10000032,22595853.0,35223874,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27
2,10000032,22595853.0,36904543,discharge,NaN,2180-05-07 17:21:27,NaN
5,10000032,22841357.0,38112554,ED,Emergency Department,2180-06-26 15:54:00,2180-06-26 21:31:00
4,10000032,22841357.0,34703856,admit,Transplant,2180-06-26 21:31:00,2180-06-27 18:49:12
3,10000032,22841357.0,34100253,discharge,NaN,2180-06-27 18:49:12,NaN
9,10000032,29079034.0,32952584,ED,Emergency Department,2180-07-22 16:24:00,2180-07-23 05:54:00
13,10000032,29079034.0,39399961,ED,Emergency Department,2180-07-23 05:54:00,2180-07-23 14:00:00
14,10000032,29079034.0,39553978,admit,Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47
10,10000032,29079034.0,35888873,transfer,Transplant,2180-07-23 23:50:47,2180-07-24 19:52:58


Hmm, strange, some transfers appear to be continuous (rows 9 with row 13) but are two separate rows for some reason. We should merge these rows at some point.

In [4]:
transfers['hadm_id'] = transfers.hadm_id.astype('Int64')

In [5]:
display(transfers.head())

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10000032,22595853,33258284,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00
1,10000032,22595853,35223874,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27
2,10000032,22595853,36904543,discharge,NaN,2180-05-07 17:21:27,NaN
3,10000032,22841357,34100253,discharge,NaN,2180-06-27 18:49:12,NaN
4,10000032,22841357,34703856,admit,Transplant,2180-06-26 21:31:00,2180-06-27 18:49:12


In [6]:
transfers.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1991704 entries, 0 to 1991703
Data columns (total 7 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   subject_id   1991704 non-null  int64 
 1   hadm_id      1644130 non-null  Int64 
 2   transfer_id  1991704 non-null  int64 
 3   eventtype    1991704 non-null  object
 4   careunit     1537380 non-null  object
 5   intime       1991704 non-null  object
 6   outtime      1537383 non-null  object
dtypes: Int64(1), int64(2), object(4)
memory usage: 108.3+ MB


In [7]:
transfers['intime'] = pd.to_datetime(transfers.intime, infer_datetime_format=True)
transfers['outtime'] = pd.to_datetime(transfers.outtime, infer_datetime_format=True)

In [8]:
transfers.nunique()

subject_id      315460
hadm_id         454324
transfer_id    1991704
eventtype            4
careunit            39
intime         1980736
outtime        1532471
dtype: int64

In [9]:
transfers_dupl = transfers.drop(columns = 'transfer_id').duplicated()
print(transfers_dupl.sum())
transfers = transfers[~transfers_dupl]

8


In [10]:
transfers[transfers.hadm_id == 22595853]

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10000032,22595853,33258284,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00
1,10000032,22595853,35223874,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27
2,10000032,22595853,36904543,discharge,NaN,2180-05-07 17:21:27,NaT


Example of an stay with three transfer events. The patient is admitted in the emergency department (event type ED, careunit Emergency Department) and stays there for 4 hours. He is then transferred to the transplant careunit, for which he needs to be admitted into hospital (eventtype admit). After 1 day he is discharged. We do not need the discharge rows, as there is nothing to be predicted from those. Rather, discharge should be the "output" for row 2. Likewise, the output for row 1 will not be ED or Emergency Department (that is a feature), but Transplant. Discharge have no duration and the resulting careunits and outtime are null values, although that is expected

In [11]:
print(transfers[transfers.outtime.isna()].nunique())

subject_id     190279
hadm_id        454321
transfer_id    454321
eventtype           1
careunit            0
intime         454236
outtime             0
dtype: int64


In [12]:
transfers.sort_values(by=['subject_id','intime'],inplace = True)
events = transfers.careunit.loc[1:].values # From second row, getting the destination careunit (class)
transfers = transfers.iloc[:-1] # Until second to last row
transfers['event'] = events

Now there should be one more nan values in the new event column than in the discharge column

In [13]:
print(sum(transfers.event.isna()))
print(sum(transfers.eventtype == 'discharge'))

454324
454323


It's correct, so we can fill the NaNs in the event column as discharges and drop the (old) discharge columns

In [14]:
transfers['event'] = transfers.event.fillna('Discharge')
transfers.dropna(axis = 0, subset = 'careunit', inplace = True)
transfers.reset_index(inplace = True,drop = True)

In [15]:
transfers.outtime.isna().sum()

0

Next step is merging transfers that are actually the same event (outtime of a transfer is equal to intime of the following transfer with same subject_ID) or that are very close together. This is because it is not useful to predict very short transfers, as these will be scheduled at least a few hours in advance. We can simplify our dataset and improve our model by mergin these rows when possible. To be conservative, we can set this lower transfer duration threshold at 3 h.

In [17]:
transfers2 = transfers.copy()

In [94]:
transfers = transfers2.copy()

In [95]:
# We first create a new column called duration, which can serve as a new feature
transfers['duration'] = (transfers.outtime - transfers.intime)/np.timedelta64(1,'h')
# We are interested in rows with duration under 3 hours...
short_transfers = (transfers['duration'] < 3).values
# And whose preceding row has the same subject_id
short_transfers *= (transfers.subject_id.shift() == transfers.subject_id).values
# And where the intime is close to the outtime of the preceding row (within 3 h)
short_transfers *= (transfers.intime < transfers.outtime.shift() + np.timedelta64(3,'h')).values
# We can edit the intime now
transfers.loc[short_transfers,'intime'] = transfers.intime[np.roll(short_transfers,-1)].values
# For eventtype and careunit we will leave the value of the row with the highest duration
shorter_inds = (transfers.duration[short_transfers] < transfers.duration[np.roll(short_transfers,-1)].values).index
# transfers.loc[shorter_inds,['eventtype','careunit']] = transfers.shift(-1).loc[shorter_inds,['eventtype','careunit']]

In [ ]:
trans

In [76]:
transfers.loc[shorter_inds,'duration'] = (transfers.outtime[shorter_inds]\
                                          - transfers.intime[shorter_inds])/np.timedelta64(1,'h')

In [83]:
# We can now drop the now duplicate rows and reset the index
transfers.drop_duplicates(subset = ['subject_id','careunit','intime'],keep = 'last',inplace = True)
transfers.reset_index(drop = True, inplace = True)

# Repeating above process
short_transfers = (transfers['duration'] < 3).values
short_transfers *= (transfers.subject_id.shift() == transfers.subject_id).values
short_transfers *= (transfers.intime < transfers.outtime.shift() + np.timedelta64(3,'h')).values
transfers.loc[short_transfers,'intime'] = transfers.intime[np.roll(short_transfers,-1)].values
shorter_inds = (transfers.duration[short_transfers] < transfers.duration[np.roll(short_transfers,-1)].values).index
transfers.loc[shorter_inds,['eventtype','careunit']] = transfers.shift(-1).loc[shorter_inds,['eventtype','careunit']]

In [93]:
transfers.shift(-1).loc[shorter_inds,['eventtype','careunit']]

,eventtype,careunit
13,ED,Emergency Department
17,ED,Emergency Department
22,ED,Emergency Department
38,ED,Emergency Department
68,transfer,Med/Surg
...,...,...
1537313,ED,Emergency Department
1537317,transfer,PACU
1537318,transfer,Discharge Lounge
1537319,transfer,Discharge Lounge


In [88]:
transfers.loc[100:105]

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,event,duration
100,10000935,26381316,37351977,transfer,Vascular,2187-08-24 15:22:52,2187-08-27 15:35:44,Discharge,72.214444
101,10000935,<NA>,37059738,ED,Emergency Department,2187-09-26 14:48:00,2187-09-26 17:50:00,Emergency Department,3.033333
102,10000935,25849114,37499077,ED,Emergency Department,2187-10-10 11:56:00,2187-10-10 19:09:46,Neurology,7.229444
103,10000935,25849114,39808296,transfer,Discharge Lounge,2187-10-10 11:56:00,2187-10-10 20:55:17,Discharge Lounge,8.988056
104,10000935,25849114,32096064,transfer,Transplant,2187-10-10 11:56:00,2187-10-10 21:11:13,Transplant,2.024167
105,10000935,25849114,37760671,transfer,Transplant,2187-10-10 21:11:13,2187-10-22 18:28:58,Hematology/Oncology,285.295833


In [97]:
transfers.loc[shorter_inds]

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,event,duration
13,10000068,25022803,31904173,admit,Emergency Department Observation,2160-03-03 21:55:00,2160-03-04 07:33:47,Discharge,1.129722
17,10000084,29888819,39560622,admit,Emergency Department Observation,2160-12-27 18:32:00,2160-12-28 16:10:39,Discharge,0.060833
22,10000108,27250926,39507710,admit,Emergency Department Observation,2163-09-27 16:18:00,2163-09-28 09:04:56,Discharge,0.015556
38,10000280,25852320,31474561,admit,Emergency Department Observation,2151-03-18 02:35:00,2151-03-18 17:04:50,Discharge,0.013889
68,10000764,27897940,37254078,transfer,Med/Surg,2132-10-14 23:32:59,2132-10-15 16:20:01,Med/Surg,0.984722
...,...,...,...,...,...,...,...,...,...
1537313,19999733,27674281,39344756,admit,Emergency Department Observation,2152-07-08 20:15:00,2152-07-09 03:51:59,Discharge,0.116389
1537317,19999784,26194817,30153747,admit,Neurology,2119-06-18 14:21:00,2119-06-18 21:48:57,Discharge Lounge,0.657778
1537318,19999784,26194817,37769396,transfer,Discharge Lounge,2119-06-18 21:09:29,2119-06-19 00:08:37,PACU,2.327778
1537319,19999784,26194817,37290592,transfer,PACU,2119-06-18 21:48:57,2119-06-19 00:25:10,Discharge Lounge,0.275833


In [99]:
transfers.loc[shorter_inds].shift(-1)

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,event,duration
13,10000084.0,29888819,39560622.0,admit,Emergency Department Observation,2160-12-27 18:32:00,2160-12-28 16:10:39,Discharge,0.060833
17,10000108.0,27250926,39507710.0,admit,Emergency Department Observation,2163-09-27 16:18:00,2163-09-28 09:04:56,Discharge,0.015556
22,10000280.0,25852320,31474561.0,admit,Emergency Department Observation,2151-03-18 02:35:00,2151-03-18 17:04:50,Discharge,0.013889
38,10000764.0,27897940,37254078.0,transfer,Med/Surg,2132-10-14 23:32:59,2132-10-15 16:20:01,Med/Surg,0.984722
68,10000886.0,21927847,30150214.0,admit,Emergency Department Observation,2178-05-08 03:40:00,2178-05-08 12:25:15,Discharge,0.654167
...,...,...,...,...,...,...,...,...,...
1537313,19999784.0,26194817,30153747.0,admit,Neurology,2119-06-18 14:21:00,2119-06-18 21:48:57,Discharge Lounge,0.657778
1537317,19999784.0,26194817,37769396.0,transfer,Discharge Lounge,2119-06-18 21:09:29,2119-06-19 00:08:37,PACU,2.327778
1537318,19999784.0,26194817,37290592.0,transfer,PACU,2119-06-18 21:48:57,2119-06-19 00:25:10,Discharge Lounge,0.275833
1537319,19999784.0,23664472,39821910.0,transfer,Hematology/Oncology Intermediate,2119-07-25 17:33:10,2119-07-30 16:21:20,Hematology/Oncology Intermediate,0.229167


In [92]:
transfers[short_transfers + np.roll(short_transfers,-1)]

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,event,duration
12,10000068,25022803,31790798,ED,Emergency Department,2160-03-03 21:55:00,2160-03-04 06:26:00,Emergency Department Observation,8.516667
13,10000068,25022803,31904173,ED,Emergency Department,2160-03-03 21:55:00,2160-03-04 07:33:47,Discharge,1.129722
16,10000084,29888819,36954971,ED,Emergency Department,2160-12-27 18:32:00,2160-12-28 16:07:00,Emergency Department Observation,21.583333
17,10000084,29888819,39560622,ED,Emergency Department,2160-12-27 18:32:00,2160-12-28 16:10:39,Discharge,0.060833
21,10000108,27250926,36533795,ED,Emergency Department,2163-09-27 16:18:00,2163-09-28 09:04:00,Emergency Department Observation,16.766667
...,...,...,...,...,...,...,...,...,...
1537317,19999784,26194817,30153747,transfer,Discharge Lounge,2119-06-18 14:21:00,2119-06-18 21:48:57,Discharge Lounge,0.657778
1537318,19999784,26194817,37769396,transfer,PACU,2119-06-18 21:09:29,2119-06-19 00:08:37,PACU,2.327778
1537319,19999784,26194817,37290592,transfer,Discharge Lounge,2119-06-18 21:48:57,2119-06-19 00:25:10,Discharge Lounge,0.275833
1537328,19999784,23664472,32607240,transfer,Surgery/Trauma,2119-07-25 17:33:10,2119-07-30 16:07:35,Hematology/Oncology Intermediate,118.573611


In [64]:
shorter_inds

Int64Index([     13,      17,      22,      38,      68,      85,      96,
                101,     109,     110,
            ...
            1537265, 1537269, 1537274, 1537277, 1537300, 1537313, 1537317,
            1537318, 1537319, 1537329],
           dtype='int64', length=189285)

In [74]:
transfers.shift(-1).loc[shorter_inds]

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,event,duration
13,10000084.0,23052089,35203156.0,ED,Emergency Department,2160-11-20 20:36:00,2160-11-21 03:20:00,Medicine,6.733333
17,10000102.0,<NA>,39891608.0,ED,Emergency Department,2136-12-20 11:49:00,2136-12-20 13:56:00,Emergency Department,2.116667
22,10000115.0,<NA>,38081480.0,ED,Emergency Department,2154-12-10 02:04:00,2154-12-10 05:59:00,Emergency Department,3.916667
38,10000280.0,<NA>,37090384.0,ED,Emergency Department,2151-12-01 14:14:00,2151-12-01 17:14:00,Emergency Department,3.000000
68,10000764.0,27897940,36796314.0,transfer,Med/Surg,2132-10-15 16:20:01,2132-10-19 17:00:08,Discharge,96.668611
...,...,...,...,...,...,...,...,...,...
1537313,19999750.0,<NA>,38224473.0,ED,Emergency Department,2144-03-22 14:27:00,2144-03-22 18:47:00,Emergency Department,4.333333
1537317,19999784.0,26194817,37769396.0,transfer,PACU,2119-06-18 21:09:29,2119-06-19 00:08:37,PACU,2.327778
1537318,19999784.0,26194817,37290592.0,transfer,Discharge Lounge,2119-06-18 21:48:57,2119-06-19 00:25:10,Discharge Lounge,0.275833
1537319,19999784.0,26194817,39890059.0,transfer,Discharge Lounge,2119-06-19 00:25:10,2119-06-19 16:10:16,Neurology,15.751667


In [75]:
transfers.loc[shorter_inds]

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,event,duration
13,10000068,25022803,31904173,ED,Emergency Department,2160-03-03 21:55:00,2160-03-04 07:33:47,Discharge,1.129722
17,10000084,29888819,39560622,ED,Emergency Department,2160-12-27 18:32:00,2160-12-28 16:10:39,Discharge,0.060833
22,10000108,27250926,39507710,ED,Emergency Department,2163-09-27 16:18:00,2163-09-28 09:04:56,Discharge,0.015556
38,10000280,25852320,31474561,ED,Emergency Department,2151-03-18 02:35:00,2151-03-18 17:04:50,Discharge,0.013889
68,10000764,27897940,37254078,transfer,Med/Surg,2132-10-14 23:32:59,2132-10-15 16:20:01,Med/Surg,0.984722
...,...,...,...,...,...,...,...,...,...
1537313,19999733,27674281,39344756,ED,Emergency Department,2152-07-08 20:15:00,2152-07-09 03:51:59,Discharge,0.116389
1537317,19999784,26194817,30153747,transfer,Discharge Lounge,2119-06-18 14:21:00,2119-06-18 21:48:57,Discharge Lounge,0.657778
1537318,19999784,26194817,37769396,transfer,PACU,2119-06-18 21:09:29,2119-06-19 00:08:37,PACU,2.327778
1537319,19999784,26194817,37290592,transfer,Discharge Lounge,2119-06-18 21:48:57,2119-06-19 00:25:10,Discharge Lounge,0.275833


In [59]:
transfers[short_transfers + np.roll(short_transfers,-1)].head(20)

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,event,duration
12,10000068,25022803,31790798,ED,Emergency Department,2160-03-03 21:55:00,2160-03-04 06:26:00,Emergency Department Observation,8.516667
13,10000068,25022803,31904173,admit,Emergency Department Observation,2160-03-04 06:26:00,2160-03-04 07:33:47,Discharge,1.129722
16,10000084,29888819,36954971,ED,Emergency Department,2160-12-27 18:32:00,2160-12-28 16:07:00,Emergency Department Observation,21.583333
17,10000084,29888819,39560622,admit,Emergency Department Observation,2160-12-28 16:07:00,2160-12-28 16:10:39,Discharge,0.060833
21,10000108,27250926,36533795,ED,Emergency Department,2163-09-27 16:18:00,2163-09-28 09:04:00,Emergency Department Observation,16.766667
22,10000108,27250926,39507710,admit,Emergency Department Observation,2163-09-28 09:04:00,2163-09-28 09:04:56,Discharge,0.015556
37,10000280,25852320,39155318,ED,Emergency Department,2151-03-18 02:35:00,2151-03-18 17:04:00,Emergency Department Observation,14.483333
38,10000280,25852320,31474561,admit,Emergency Department Observation,2151-03-18 17:04:00,2151-03-18 17:04:50,Discharge,0.013889
67,10000764,27897940,36224103,admit,Emergency Department Observation,2132-10-14 23:32:59,2132-10-15 15:20:56,Med/Surg,15.799167
68,10000764,27897940,37254078,transfer,Med/Surg,2132-10-15 15:20:56,2132-10-15 16:20:01,Med/Surg,0.984722


In [252]:
shorter_transfers = short_transfers * (transfers.duration.shift() <= transfers.duration).values

In [254]:
transfers[shorter_transfers]

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,event,duration
153,10001180,27864856,33254822,transfer,Labor & Delivery,2197-06-06 14:32:20,2197-06-06 17:48:32,Obstetrics Postpartum,1.877778
197,10001401,24818636,35887732,transfer,Discharge Lounge,2131-07-30 21:41:04,2131-07-31 00:00:55,Hematology/Oncology,2.218611
215,10001523,<NA>,37146432,ED,Emergency Department,2120-12-30 16:57:00,2121-01-01 09:34:00,Emergency Department,2.350000
304,10002012,<NA>,38333639,ED,Emergency Department,2167-03-17 12:37:00,2168-09-07 21:14:00,Vascular,1.433333
370,10002221,21008195,37843288,transfer,Discharge Lounge,2200-09-29 17:28:51,2200-09-29 19:37:42,Medicine,1.933333
...,...,...,...,...,...,...,...,...,...
1537157,19998497,25178709,31223688,transfer,Medicine,2140-08-29 18:12:52,2140-08-29 18:32:39,Medicine/Cardiology,0.211944
1537158,19998497,25178709,32415917,transfer,Medicine/Cardiology,2140-08-29 18:19:56,2140-08-29 18:55:46,Medicine/Cardiology,0.385278
1537176,19998591,23299044,31473178,transfer,Surgery/Trauma,2185-03-10 15:29:21,2185-03-10 16:58:13,Surgery/Trauma,1.327222
1537306,19999625,25304202,35596911,ED,Emergency Department,2139-01-03 13:10:00,2139-10-10 19:18:00,Medical/Surgical Intensive Care Unit (MICU/SICU),2.566667


Some of the preceding rows may be short themselves. We are only going to edit those rows that are shorter than their preceding row. 

In [255]:
transfers.loc[short_transfers]

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,event,duration
13,10000068,25022803,31904173,admit,Emergency Department Observation,2160-03-03 21:55:00,2160-03-04 07:33:47,Discharge,1.129722
17,10000084,29888819,39560622,admit,Emergency Department Observation,2160-12-27 18:32:00,2160-12-28 16:10:39,Discharge,0.060833
22,10000108,27250926,39507710,admit,Emergency Department Observation,2163-09-27 16:18:00,2163-09-28 09:04:56,Discharge,0.015556
24,10000115,<NA>,30295111,ED,Emergency Department,2154-12-10 02:04:00,2154-12-17 16:59:00,Emergency Department,0.366667
30,10000117,<NA>,30632130,ED,Emergency Department,2181-11-15 08:15:53,2183-07-17 11:31:00,Emergency Department,1.016667
...,...,...,...,...,...,...,...,...,...
1537313,19999733,27674281,39344756,admit,Emergency Department Observation,2152-07-08 20:15:00,2152-07-09 03:51:59,Discharge,0.116389
1537317,19999784,26194817,30153747,admit,Neurology,2119-06-18 14:21:00,2119-06-18 21:48:57,Discharge Lounge,0.657778
1537318,19999784,26194817,37769396,transfer,Discharge Lounge,2119-06-18 21:09:29,2119-06-19 00:08:37,PACU,2.327778
1537319,19999784,26194817,37290592,transfer,PACU,2119-06-18 21:48:57,2119-06-19 00:25:10,Discharge Lounge,0.275833


In [256]:
transfers[short_transfers + np.roll(short_transfers,-1)]

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,event,duration
12,10000068,25022803,31790798,ED,Emergency Department,2160-03-03 21:55:00,2160-03-04 06:26:00,Emergency Department Observation,8.516667
13,10000068,25022803,31904173,admit,Emergency Department Observation,2160-03-03 21:55:00,2160-03-04 07:33:47,Discharge,1.129722
16,10000084,29888819,36954971,ED,Emergency Department,2160-12-27 18:32:00,2160-12-28 16:07:00,Emergency Department Observation,21.583333
17,10000084,29888819,39560622,admit,Emergency Department Observation,2160-12-27 18:32:00,2160-12-28 16:10:39,Discharge,0.060833
21,10000108,27250926,36533795,ED,Emergency Department,2163-09-27 16:18:00,2163-09-28 09:04:00,Emergency Department Observation,16.766667
...,...,...,...,...,...,...,...,...,...
1537317,19999784,26194817,30153747,admit,Neurology,2119-06-18 14:21:00,2119-06-18 21:48:57,Discharge Lounge,0.657778
1537318,19999784,26194817,37769396,transfer,Discharge Lounge,2119-06-18 21:09:29,2119-06-19 00:08:37,PACU,2.327778
1537319,19999784,26194817,37290592,transfer,PACU,2119-06-18 21:48:57,2119-06-19 00:25:10,Discharge Lounge,0.275833
1537328,19999784,23664472,32607240,transfer,Surgery/Trauma,2119-07-25 17:33:10,2119-07-30 16:07:35,Hematology/Oncology Intermediate,118.573611


In [257]:
shorter_inds = transfers[short_transfers][shorter_transfers].index
longer_inds = transfers[np.roll(short_transfers,-1)][shorter_transfers].index

transfers.loc[shorter_inds,['careunit','intime']] = transfers.loc[longer_inds,['careunit','intime']].values

ValueError: could not broadcast input array from shape (8212,) into shape (32305,)

In [ ]:
transfers.loc[shorter_inds & longer_inds]

Now we can drop any duplicate rows with the same subject_id, careunit, intime and outtime, recalculate duration and repeat this step

In [ ]:
transfers.drop_duplicates(subset = ['subject_id','careunit','intime'],keep = 'last',inplace = True)
transfers.sort_values(by = ['subject_id','intime'],inplace = True)
transfers.reset_index(drop = True, inplace = True)
transfers['duration'] = (transfers.outtime - transfers.intime)/np.timedelta64(1,'h')

# Recalculating booleans
short_transfers = (transfers['duration'] < 3).values
short_transfers *= (transfers.subject_id.shift() == transfers.subject_id).values
shorter_transfers = transfers.duration[short_transfers].values < transfers.duration[np.roll(short_transfers,-1)].values
shorter_inds = transfers[short_transfers][shorter_transfers].index
longer_inds = transfers[np.roll(short_transfers,-1)][shorter_transfers].index

In [ ]:
sum(transfers.duplicated(subset = ['subject_id','careunit','intime']))

In [ ]:
transfers2[transfers2.subject_id == 10000935]

In [ ]:
transfers[short_transfers + np.roll(short_transfers,-1)]

In [ ]:
transfers[short_transfers][~shorter_transfers]

In [ ]:
sum(short_transfers)

In [ ]:
print('Does any of these rows have a shorter duration than the preceding row?',\
      transfers.duration[np.roll(short_transfers,-1)].values < transfers.duration[short_transfers].values)

In [ ]:
transfers.loc[short_transfers + np.roll(short_transfers,-1)] 

In [ ]:
transfers[short_transfers + np.roll(short_transfers,-1)]
tramsfers[short_transfers

In [ ]:
transfers

In [ ]:
transfers2 = transfers.copy()

So not all the rows here are actually rows that have been split, as sometimes the first row is followed by a row of a different patient. We will have to remove these rows as a transfer to the same careunit does not make much sense

In [ ]:
split_rows = transfers[['subject_id','outtime','event']] == transfers[['subject_id','intime','careunit']]\
.shift(-1).rename(columns = {'intime':'outtime','careunit':'event'})
split_rows = split_rows.all(axis = 1)
split_rows = split_rows & (transfers.event == transfers.careunit)

In [ ]:
transfers[['outtime','event']] = transfers[['outtime','event']].where(~split_rows,transfers[['outtime','event']].shift(-1))

non_splits = ~split_rows.shift().fillna(False)
transfers = transfers[non_splits]

Let's repeat to make sure that we have taken care of all the split rows (maybe some were split into 3 or more)

In [ ]:
split_rows = transfers[['subject_id','outtime','event']] == transfers[['subject_id','intime','careunit']]\
.shift(-1).rename(columns = {'intime':'outtime','careunit':'event'})
split_rows = split_rows.all(axis = 1)
split_rows = split_rows & (transfers.event == transfers.careunit)
print(split_rows.any())

Now we can check which events and careunits are the same without being continuous transfers

In [ ]:
non_transfers = transfers.careunit == transfers.event
display(transfers[non_transfers])
print(transfers[non_transfers].info())
print(transfers[non_transfers].describe(include = 'all',datetime_is_numeric=True))
print((transfers.outtime[non_transfers]-transfers.intime[non_transfers]).describe())

We can use these for prediction of next department, so we will drop these row. They could be useful only if planning to add number or duration of previous transfers, but we are not planning to add these features for now. We will drop these rows.

In [258]:
transfers = transfers[~non_transfers]
transfers.info()

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

We have decreased the number of our rows by a third. We can make sure that there are no discharge events that are immediately followed by an admission/transfer into another department, as these should not be classed as discharge.

In [ ]:
transfers[transfers.subject_id == 15526381].sort_values(by = ['subject_id','intime'])

In [ ]:
discharges_bool.shift(1)

In [ ]:
intimes

In [ ]:
outtimes

In [ ]:
disch_bool = transfers.event == 'Discharge'

outtimes = transfers.loc[disch_bool,['subject_id','outtime']]\
    .rename(columns = {'outtime':'intime'}).iloc[:-1].reset_index(drop = True)
intimes = transfers.loc[disch_bool.shift().fillna(False),['subject_id','intime']]\
    .reset_index(drop = True)

In [ ]:
tal = (outtimes.values == intimes.values)
cual = [l[0]*l[1] for l in tal]

In [ ]:
sum(cual)

In [ ]:
non_disch = (outtimes == intimes).all(axis = 1)
print(f'There are {sum(non_disch)} fake discharges')

In [ ]:
np.append(non_disch.values,False)

In [ ]:
transfers.loc[transfers.event == 'Discharge'][np.append(non_disch.values,False)]

Let's start merging tables. We can begin with the patients table, which contains information on age and gender of patient

In [54]:
# Caca
cat = list('ABCDGHJLMNPRSV')
cat.extend(['pres_no'])

dfl = transfers[['subject_id','intime','outtime','transfer_id']].copy()
dfr = tranfers[cat].copy() # includes starttime

def step_merger(dfl,dfr,n = 50):
    
    df_merged = pd.merge(dfl.head(0),dfr.head(0), how = 'left')

    non_dums = list(df_merged.columns.values)
    [non_dums.remove(col) for col in dums]
    dums = copy.deepcopy(dums)
    dums.append(index_col)

    df_merged.drop(columns = index_col,inplace = True)
    df_merged.index.rename(index_col,inplace = True)

    dfl.sort_values(by = on_col,inplace=True)
    dfl.reset_index(inplace = True, drop = True)
    dfr.sort_values(by = on_col,inplace=True)
    dfr.reset_index(inplace = True, drop = True)

    ind_l1 = 0
    ind_r1 = 0

    inc = round(len(dfr)/n)
    max_l = dfl[on_col].max()
    max_r = dfr[on_col].max()
    for i in range(n):
        # Taking first lim rows 
        max_oncol = dfr.loc[ind_r1:].head(inc)[on_col].values[-1]

        if max_oncol < max_l and max_oncol < max_r:
            ind_r2 = (dfr[on_col]> max_oncol).argmax() - 1
            dfr_chunk = dfr.loc[ind_r1:ind_r2]

            ind_l2 = dfl[dfl[on_col] == dfr_chunk[on_col].max()].index.values[-1]

            dfl_chunk = dfl.reset_index(drop = True).loc[ind_l1:ind_l2]

        else:
            dfr_chunk = dfr.loc[ind_r1:]
            dfl_chunk = dfl.loc[ind_l1:]

        df_merging = pd.merge(dfl_chunk,dfr_chunk, on = on_col, how = 'left')

        valid_rows = (df_merging.starttime>df_merging.intime) & (df_merging.starttime<df_merging.outtime)
        df_merging.loc[~valid_rows,dums[:-1]] = 0

        df_merging = pd.concat([df_merging[dums].groupby(index_col).sum().astype(int),
                                df_merging[non_dums].groupby(index_col).first()],
                                axis = 1)
        df_merged = pd.concat([df_merged,df_merging], axis = 0)
        ind_r1 = ind_r2 + 1
        ind_l1 = ind_l2 + 1
        print(f'Loop {i + 1} out of {n}')
    
    df_merged.reset_index(drop = False, inplace = True)
    return df_merged

In [259]:
patients = pd.read_csv('../Capstone data/MIMIC IV/patients.csv')
patients.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,NaN


In [260]:
patients['dod'] = pd.to_datetime(patients.dod, infer_datetime_format = True)

Basic eda:

In [261]:
print(patients.info(show_counts = True))
print('\n')
print(patients.dod.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315460 entries, 0 to 315459
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   subject_id         315460 non-null  int64         
 1   gender             315460 non-null  object        
 2   anchor_age         315460 non-null  int64         
 3   anchor_year        315460 non-null  int64         
 4   anchor_year_group  315460 non-null  object        
 5   dod                30636 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 14.4+ MB
None


count                   30636
unique                  19705
top       2129-11-19 00:00:00
freq                        7
first     2104-12-24 00:00:00
last      2212-01-22 00:00:00
Name: dod, dtype: object


Okay, no null values (other that the patients that did not die) and nothing weird on date of death (dod). Let's look at info on gender and age.

In [262]:
pd.Series(patients.gender.isna().mean()*100)

0    0.0
dtype: float64

In [263]:
nachopy.display_sbs(
                [patients.gender.value_counts(normalize = True)*100,\

                patients.loc[~patients.dod.isna(),'gender'].value_counts()/patients.gender.value_counts()*100,\

                pd.concat([patients.anchor_age[patients.gender == 'M'].describe().rename('Males'),\
                patients.anchor_age[patients.gender == 'F'].describe().rename('Females')], axis = 1)],\

                titles = ['Patient genders', '% that died', 'Male and female age distributions']
)

Patient genders 
 
 
   
 gender 
 
 
 
 
 F 
 52.906549 
 
 
 M 
 47.093451 
 
 
 

 
 % that died 
 
 
   
 gender 
 
 
 
 
 F 
 8.775367 
 
 
 M 
 10.763255 
 
 
 

 
 Male and female age distributions 
 
 
   
 Males 
 Females 
 
 
 
 
 count 
 148561.000000 
 166899.000000 
 
 
 mean 
 49.026588 
 48.080480 
 
 
 std 
 20.146493 
 21.515040 
 
 
 min 
 18.000000 
 18.000000 
 
 
 25% 
 30.000000 
 28.000000 
 
 
 50% 
 49.000000 
 46.000000 
 
 
 75% 
 65.000000 
 65.000000 
 
 
 max 
 91.000000 
 91.000000

Okay, good for now. We do not have any death registered in our transfers tables. This should be an event different from discharge, as the features for patients that expired and patients that were discharged from the hospital will probably be significantly different. Now we are going to join gender, age and dod.

In [264]:
transfers = pd.merge(transfers,patients[['subject_id','gender','anchor_age','dod']], on = 'subject_id',how = 'left')
transfers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537372 entries, 0 to 1537371
Data columns (total 12 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   subject_id   1537372 non-null  int64         
 1   hadm_id      1189806 non-null  Int64         
 2   transfer_id  1537372 non-null  int64         
 3   eventtype    1537372 non-null  object        
 4   careunit     1537372 non-null  object        
 5   intime       1537372 non-null  datetime64[ns]
 6   outtime      1537372 non-null  datetime64[ns]
 7   event        1537372 non-null  object        
 8   duration     1537372 non-null  float64       
 9   gender       1537372 non-null  object        
 10  anchor_age   1537372 non-null  int64         
 11  dod          343945 non-null   datetime64[ns]
dtypes: Int64(1), datetime64[ns](3), float64(1), int64(3), object(4)
memory usage: 153.9+ MB


In [265]:
# deceased = transfers.dod == transfers.outtime.dt.date
# print('Did any patients die at event "Discharge"?',deceased.any())
# print('How many?', deceased.sum(),'And the percentage?', round(deceased.mean()*100,2))
# transfers[deceased].head()
# transfers[['gender','anchor_age']].isna().mean()
# transfers.loc[deceased,'event'] = 'Deceased'
# transfers.drop(columns = 'dod',inplace = True)
# print('NaN values for new columns')
# print(transfers[['gender','anchor_age']].isna().mean())

Less than 1% of patients in the transfers column died rather than get discharged. We will change the class to "Deceased" for these rows. We can drop the dod column and check for any missing values in the new columns. 

Okay, clean df again. Now let's check what we can add from admissions (private info not shown)

In [266]:
admissions = pd.read_csv('../Capstone data/MIMIC IV/admissions.csv')
# display(admissions.head())
print(admissions.info(show_counts = True))
print(admissions.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454324 entries, 0 to 454323
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   subject_id            454324 non-null  int64 
 1   hadm_id               454324 non-null  int64 
 2   admittime             454324 non-null  object
 3   dischtime             454324 non-null  object
 4   deathtime             9087 non-null    object
 5   admission_type        454324 non-null  object
 6   admission_location    454324 non-null  object
 7   discharge_location    328519 non-null  object
 8   insurance             454324 non-null  object
 9   language              454324 non-null  object
 10  marital_status        444608 non-null  object
 11  race                  454324 non-null  object
 12  edregtime             315460 non-null  object
 13  edouttime             315460 non-null  object
 14  hospital_expire_flag  454324 non-null  int64 
dtypes: int64(3), obje

Okay, let's do a basic EDA of some variables that could be interesting

In [267]:
columns = ['admission_type','discharge_location','insurance','language','marital_status','race']
nachopy.display_sbs([admissions[col].value_counts() for col in columns],max_rows = 17, suffix = 'value_counts')

,admission_type
EW EMER.,157487
EU OBSERVATION,100133
OBSERVATION ADMIT,55210
URGENT,47127
SURGICAL SAME DAY ADMISSION,35994
DIRECT EMER.,20584
DIRECT OBSERVATION,19698
ELECTIVE,11089
AMBULATORY OBSERVATION,7002
,discharge_location


Admission type seems quite relevant, we could look into doing some label encoding. Discharge location also looks relevant and should probably be used to update the class ("event"). Insurance relevant as well, as medicaid and medicare patients could have different outcomes than privately-insured. English and marital status will be added for now, although probably little relevance. Race may be relevant, could be added simply as a dummy column (1 if white, 0 otherwise). We will join now and adjust later during the EDA. 

In [268]:
transfers['hadm_id_nans'] = transfers.hadm_id
transfers.loc[transfers.hadm_id.isna(),'hadm_id'] =  transfers.index[transfers.hadm_id.isna()] # Just to try and fill most of them
columns.append('hadm_id')
transfers = pd.merge(transfers,admissions[columns], how = 'left', on = 'hadm_id')
transfers.drop(columns = 'hadm_id',inplace = True)
transfers.rename(columns = {'hadm_id_nans':'hadm_id'},inplace = True)
transfers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537372 entries, 0 to 1537371
Data columns (total 18 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   subject_id          1537372 non-null  int64         
 1   transfer_id         1537372 non-null  int64         
 2   eventtype           1537372 non-null  object        
 3   careunit            1537372 non-null  object        
 4   intime              1537372 non-null  datetime64[ns]
 5   outtime             1537372 non-null  datetime64[ns]
 6   event               1537372 non-null  object        
 7   duration            1537372 non-null  float64       
 8   gender              1537372 non-null  object        
 9   anchor_age          1537372 non-null  int64         
 10  dod                 343945 non-null   datetime64[ns]
 11  hadm_id             1189806 non-null  Int64         
 12  admission_type      1189806 non-null  object        
 13  discharge_lo

First, let's see if there are any subject_id that has a hadm_id on the admissions table but not on the transfers table. For race insurance and language there are no missing values on the admissions table, so we can try to retrieve them by updating using subject_id for the first occurence of these values

In [269]:
columns.append('subject_id')
missing_rows = admissions[columns].groupby('subject_id').first()

transfers.reset_index(drop = False)
transfers.set_index('subject_id',drop = False,inplace = True)
transfers.update(missing_rows)
transfers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537372 entries, 10000032 to 19999987
Data columns (total 18 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   subject_id          1537372 non-null  int64         
 1   transfer_id         1537372 non-null  int64         
 2   eventtype           1537372 non-null  object        
 3   careunit            1537372 non-null  object        
 4   intime              1537372 non-null  datetime64[ns]
 5   outtime             1537372 non-null  datetime64[ns]
 6   event               1537372 non-null  object        
 7   duration            1537372 non-null  float64       
 8   gender              1537372 non-null  object        
 9   anchor_age          1537372 non-null  int64         
 10  dod                 343945 non-null   datetime64[ns]
 11  hadm_id             1366352 non-null  object        
 12  admission_type      1366352 non-null  object        
 13  disc

We have increased our number of hadm_ids and also the other new columns, which is good. We could now try to fill the missing values using values from the table itself.

No luck, we have only filled a few marital_status values (which are the most liable to change from the columns). We can maybe fill some features using the edstays from ED, let's try that

In [270]:
edstays = pd.read_csv('../Capstone data/MIMIC IV/ed/edstays.csv')
edstays.head().drop(columns = ['race','gender']) # We will not show sensitive information that can help reidentify patients

,subject_id,hadm_id,stay_id,intime,outtime,arrival_transport,disposition
0,10000032,22595853.0,33258284,2180-05-06 19:17:00,2180-05-06 23:30:00,AMBULANCE,ADMITTED
1,10000032,22841357.0,38112554,2180-06-26 15:54:00,2180-06-26 21:31:00,AMBULANCE,ADMITTED
2,10000032,25742920.0,35968195,2180-08-05 20:58:00,2180-08-06 01:44:00,AMBULANCE,ADMITTED
3,10000032,29079034.0,32952584,2180-07-22 16:24:00,2180-07-23 05:54:00,AMBULANCE,HOME
4,10000032,29079034.0,39399961,2180-07-23 05:54:00,2180-07-23 14:00:00,AMBULANCE,ADMITTED


The ED collects different data than the hospital, which is unfortunate. We will leave these features as they are for now.carry out analysis regardless. It appears we can add race (we can just group by subject_id and then update the column with the first occurence.

In [271]:
missing_rows = edstays[['hadm_id','race','subject_id']].groupby('subject_id').first()
transfers.update(missing_rows)
transfers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537372 entries, 10000032 to 19999987
Data columns (total 18 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   subject_id          1537372 non-null  int64         
 1   transfer_id         1537372 non-null  int64         
 2   eventtype           1537372 non-null  object        
 3   careunit            1537372 non-null  object        
 4   intime              1537372 non-null  datetime64[ns]
 5   outtime             1537372 non-null  datetime64[ns]
 6   event               1537372 non-null  object        
 7   duration            1537372 non-null  float64       
 8   gender              1537372 non-null  object        
 9   anchor_age          1537372 non-null  int64         
 10  dod                 343945 non-null   datetime64[ns]
 11  hadm_id             1366352 non-null  object        
 12  admission_type      1366352 non-null  object        
 13  disc

In [272]:
transfers.transfer_id.nunique()

1537372

Now let's merge with prescriptions

In [273]:
prescriptions = pd.read_pickle('../Capstone data/pickles/prescriptions_final.pkl')

In [274]:
cat = list(prescriptions.drop(columns = ['subject_id','hadm_id','starttime']).columns)
prescriptions[prescriptions.A<0] = 0

prescriptions['pres_number'] = 1
cat.append('pres_number')

In [275]:
transfers2 = transfers.copy()

In [ ]:
# Initialising variables for loop
# Getting only necessary columns for the row selection
dfl = transfers[['subject_id','intime','outtime','transfer_id']].set_index('subject_id').sort_index()
# Adding hadm_id because it will be useful to fill hadm_id column on transfers
dfr = prescriptions.drop(columns = 'hadm_id').set_index('subject_id').sort_index() 

n = 15
nums = copy.deepcopy(cat)
non_nums = ['subject_id','intime','outtime','starttime']

df_merged = pd.DataFrame(columns = nums)

len_r = len(dfr)
max_l = dfl.index.max()
max_r = dfr.index.max()

inc = round(len(dfr)/n)
ind_r = copy.copy(inc)
sub_1 = 0
sub_2 = 0

for i in range(n):
    
    ind_r += inc 
    if ind_r < len_r:
        sub_2 = dfr.iloc[ind_r].name # Getting last index in increment 
    else:
        sub_2 = max_l # Getting last subject on left (transfers) df when reaching the end of the df
    
    dfr_chunk = dfr.loc[sub_1:sub_2]
    dfl_chunk = dfl.loc[sub_1:sub_2]
    # Inner join as any rows without prescriptions are lost anyway (will be recovered later)
    df_merging = pd.merge(dfl_chunk,dfr_chunk, on = 'subject_id') 

    # Selecting only rows where the prescription is between starttime and outtime 
    valid_rows = (df_merging.starttime>=df_merging.intime) & (df_merging.starttime<df_merging.outtime)
    
    df_merging = df_merging[valid_rows].reset_index(drop = False) # Putting subject_id away from index and back in columns
    # Retrieving hadm_id from prescriptions to merge later
    df_merging = df_merging.groupby('transfer_id')[nums].sum() # We sum to frequency encode prescriptions

    df_merged = pd.concat([df_merged, df_merging],axis = 0)
    
    sub_1 = sub_2 + 1
    
    print(f'Loop {i + 1}/{n}',end='\r')
print()

transfers.set_index('transfer_id',inplace = True)
transfers = transfers.join(df_merged)

In [ ]:
transfers.info()

In [ ]:
transfers[cat] = transfers[cat].fillna(0)
transfers[cat] = transfers[cat].astype('uint16')
transfers['anchor_age'] = transfers.anchor_age.astype('uint8')
transfers['hadm_id'] = transfers.hadm_id.astype('Int64')

We are done with all the merging so for now, we will leave transfers_id as index, as it is a unique identifier of the rows. We will also save the df in our pickle jar as version 1 of the merging. In this pickle we will do an EDA and the feature selection. 

In [ ]:
transfers.to_pickle('../Capstone data/pickles/transfers_v1.pkl')

We filled around half of the remaining missing values for race. Now let's merge with prescriptions

Dropping rows with unknown

In [ ]:
fig1 = plt.figure(figsize = (8,8))
presperevent.sort_values().plot.barh()
plt.title('Patients that ended up in oncology, cardiology and ICU-related\ndepartments had many more prescriptions in previous transfer',pad = 15)
plt.xlabel('Number of prescriptions per transfer')
fig1.savefig('prespertransfervsunit.svg',bbox_inches='tight')

In [ ]:
fig2 = plt.figure(figsize = (10,8))
sns.heatmap(catsperevent.divide(presperevent, axis = 0)*100,cmap='viridis',xticklabels= True)
plt.title('Patients that ended up in different departments were prescribed\ndifferent types of drugs',pad = 15)
plt.xlabel('Number of prescriptions per transfer')
fig2.savefig('drugtypesvcareunit.svg',bbox_inches='tight')

In [ ]:
cat = list('ABCDGHJLMNPRSV')
cat.append('event')
catsperevent = transfers[cat].groupby('event')
catsperevent = catsperevent.sum()/catsperevent.count().sort_index()
presperevent = catsperevent.sum(axis = 1).sort_index(ascending = False)

fig,_ = plt.subplots(2,1,figsize = (8,15))

plt.subplot(2,1,1)
presperevent.sort_values().plot.barh()

plt.subplot(2,1,2)
sns.heatmap(catsperevent.divide(presperevent, axis = 0)*100,cmap='viridis')

plt.show()


In [ ]:
cat = list('ABCDGHJLMNPRSV')
cat.append('event')
catsperevent = transfers[cat].groupby('event')
catsperevent = catsperevent.sum()/catsperevent.count()

Okay, heatmap probably relevant, but I need to normalise the number of prescriptions per event by the duration of the stay

In [ ]:
tran

In [ ]:
poe = pd.read_csv('../Capstone data/MIMIC IV/poe.csv')

In [ ]:
transfers[cat]

In [ ]:
poe.order_type.value_counts(normalize = True)


In [ ]:
cap_utils.display_sbs(['Order subtype value counts'],poe.order_subtype.value_counts(normalize = True))

Probably dropping medications? Could add order sub type and filling the empty values (loads) with the simple order type?

In [ ]:
ed_dupl = edstays.drop(columns = 'stay_id').duplicated()
print(ed_dupl.sum())
edstays = edstays[~ed_dupl]

In [ ]:
edstays.info()

In [ ]:
uniqueid_set = set(transfers.hadm_id.unique())
hadm_len = len(uniqueid_set)
unique_stayid = edstays.stay_id.unique()
stay_len = len(unique_stayid)
set_len = 0
i = 0
while (len(uniqueid_set) > set_len) & (i<stay_len):
    set_len = len(uniqueid_set)
    uniqueid_set.add(unique_stayid[i])
    i +=1
if set_len == hadm_len + stay_len-1:
    print('All good')
else:
    print('caca')

In [ ]:
edstays = edstays[['subject_id','stay_id','intime','outtime']]

In [ ]:
edstays.info()

In [ ]:
edstays['intime'] = pd.to_datetime(edstays.intime,infer_datetime_format=True)
edstays['outtime'] = pd.to_datetime(edstays.outtime,infer_datetime_format=True)

In [ ]:
ed_dupl = edstays.drop(columns = 'stay_id').duplicated()
print(ed_dupl.sum())
edstays = edstays[~ed_dupl]

In [ ]:
transfers = transfers.merge(
                        edstays,
                        left_on = ['subject_id','intime','outtime'], 
                        right_on = ['subject_id','intime','outtime'],
                        how = 'left',
                        copy = False)

In [ ]:
transfers.loc[transfers.hadm_id.isna(),'hadm_id'] = transfers.loc[transfers.hadm_id.isna(),'stay_id']
sum(transfers.hadm_id.isna())

In [ ]:
transfers.careunit.value_counts()

In [ ]:
cat_pres.A.value_counts(normalize = True)*100

In [ ]:
unclass_drugs = cat_pres.drug[cat_pres.A<0].drop_duplicates()
udrugs_gsnvc = cat_pres.gsn[cat_pres.A<0].value_counts()
gsn_udrugs = udrugs_gsnvc[udrugs_gsnvc>1].index.values

In [ ]:
cat.append('gsn')
cat_gsn = cat_pres.loc[cat_pres.gsn.isin(gsn_udrugs),cat].groupby('gsn').max()
cat_gsn = cat_gsn[cat_gsn.A>-1]
cat_gsn

In [ ]:
cat = cat[:-1]
cat_pres[cat] = cat_pres[cat].where(cat_pres[cat]>=0)

In [ ]:
cat_pres.set_index(ind,append = True)

In [ ]:
cat_pres.set_index('gsn', inplace = True)
cat_pres.update(cat_gsn)
cat_pres.set_index(ind)

In [ ]:
cat_pres.head()

In [ ]:
sum(cat_pres.drug.str.contains('polyvalent'))

In [ ]:
cat_pres.gsn[misspelt_drugs_bool].value_counts().describe()

In [ ]:
(5303415+7791711)/len(cat_pres)

In [ ]:
cat

In [ ]:
cat = list('ABCDGHJLMNPRSV')
cat.append('drug')
cat_drug = cat_pres[cat].fillna(0.0001).groupby('formulary_drug_cd',dropna = False).sum()
cat_drug

In [ ]:
atr4_pres.atc4_name.nunique()

The missing values labelled as 0 are now empty strings due to the conversion. There may be other values that are not 0, maybe even some could be in other ndc formats (hyphenated or with less digits). We can find out how many rows have which number of digits.

So all fields that do not have 9-digit numbers have no digits. We can consider all of these as missing values.

More than 12% of ndc fields are missing. Some of the drugs listed here may not have an ndc (Sodium Chloride flush on 2nd row for instance), but others may be mistakes. NDC is specific to product, not to drug (same drug from different companies will have a different code) so filling from the table would not be very accurate. Given that what we need the NDC just to classify our drugs, we will leave these fields blank and will fill the categories instead after NDC-matching.


In [ ]:
prescriptions.ndc[~missing_ndc].drop_duplicates().to_csv('Unique_NDCs.csv')

Let us see if we can fill these missing values by using the other fields in the table. pharmacy_id, gsn, drug and formulary_drug_cd could be used for this.

In [ ]:
prescriptions.formulary_drug_cd.info()

In [ ]:
fdrugna = prescriptions[prescriptions.formulary_drug_cd.isna()]

In [ ]:
def df_filler(x):
    

prescriptions.formulary_drug_cd[prescriptions.drug.isin(fdrugna.drug)].isna()

In [ ]:
prescriptions.gsn.value_counts()

In [ ]:
prescriptions.pharmacy_id.value_counts()

In [ ]:
prescriptions.formulary_drug_cd.value_counts()

In [ ]:
prescriptions.formulary_drug_cd.info()

In [ ]:
prescriptions.drug = prescriptions.drug.str.lower()
prescriptions.drug.value_counts().describe()

In [ ]:
ndc_bool = prescriptions.ndc!=u'' & prescriptions 
print(sum(ndc_bool))
print(sum(~ndc_bool))

In [ ]:
elswhr_bool = prescriptions.loc[~ndc_bool,'drug'].isin(prescriptions.loc[ndc_bool,'drug'])
print(round(sum(elswhr_bool)/sum(~ndc_bool)*100,2))

In [ ]:
prescriptions.loc[elswhr_bool,'ndc'] = prescriptions.loc[ndc_bool*prescriptions.,'ndc']

In [ ]:
prescriptions.ndc.nunique()

In [ ]:
import numpy as np

In [ ]:
medication = pd.read_json(r'C:\Users\Nacho\Downloads\Medication.ndjson', lines = True)

In [ ]:
medication.shape

In [ ]:
prescriptions.ndc.head()

In [ ]:
ndc_desc = pd.read_csv("G:\My Drive\Brainstation\Capstone project\product.csv",usecols = ['PRODUCTNDC','NONPROPRIETARYNAME','PHARM_CLASSES'])

In [ ]:
ndc_desc = ndc_desc.loc[~ndc_desc.PHARM_CLASSES.isna()].reset_index(drop = True)

In [ ]:
ndc_desc.head()

In [ ]:
def sht2nineNDC(x):
    x = x.split('-')
    x = x[0].zfill(5) + x[1].zfill(4)
    
    return x
    

ndc_desc.PRODUCTNDC = ndc_desc.PRODUCTNDC.apply(sht2nineNDC)

In [ ]:
ndc_desc.PRODUCTNDC.head()

In [ ]:
classpername = ndc_desc[['NONPROPRIETARYNAME','PHARM_CLASSES']].groupby('NONPROPRIETARYNAME').nunique()

In [ ]:
classperndc = ndc_desc[['PRODUCTNDC','PHARM_CLASSES']].groupby('PRODUCTNDC').nunique()

In [ ]:
classpername.value_counts()

In [ ]:
classperndc.value_counts()

In [ ]:
np.mean(prescriptions.ndc.isna())*100

In [ ]:
prescriptions.loc[prescriptions.ndc.isna()].head(10)

In [ ]:
ndc_desc.rename(columns = {'PRODUCTNDC' : 'ndc'},inplace = True)

In [ ]:
presfinal = pd.merge(prescriptions,ndc_desc, how = 'left',on = 'ndc')

In [ ]:
np.mean(presfinal.PHARM_CLASSES.isna())

In [ ]:
np.mean(presfinal.NONPROPRIETARYNAME.isna())

In [ ]:
presfinal[['ndc','drug','NONPROPRIETARYNAME','PHARM_CLASSES']]

In [ ]:
presfinal.hadm_id.nunique()

In [ ]:
len(presfinal.PHARM_CLASSES.isna())-sum(presfinal.PHARM_CLASSES.isna())

In [ ]:
classperhadm = presfinal[['PHARM_CLASSES','hadm_id']].groupby('hadm_id').nunique()
classperhadm.describe()

In [ ]:
classperhadm.loc[classperhadm.PHARM_CLASSES == 0].count()

In [ ]:
prescriptions.ndc.apply(lambda x: str(x)[:-2].zfill(9))

In [ ]:
print(ndc_desc.NONPROPRIETARYNAME.nunique())
print(ndc_desc_clean.NONPROPRIETARYNAME.nunique())

In [ ]:
ndc_desc.shape

In [ ]:
ndc_desc.PRODUCTNDC.apply(lambda x: int(x.replace('-','')))

In [ ]:
ndc_desc.PHARM_CLASSES.value_counts

In [ ]:
poe = pd.read_csv('MIMIC IV ED/poe.csv')

In [ ]:
poe.order_status.value_counts(normalize = True)

In [ ]:
poe.loc[poe.order_type == 'Lab'].order_subtype.value_counts()

In [ ]:
poe.order_type.value_counts()

In [ ]:
admissions = pd.read_csv('MIMIC IV ED/admissions.csv')

In [ ]:
admissions.race.value_counts(normalize = True)

In [ ]:
sum(admissions.admission_location.isna())

In [ ]:
admissions.admission_location.value_counts(normalize = True)

In [ ]:
transfers = pd.read_csv('MIMIC IV ED/transfers.csv')

In [ ]:
sum(transfers.eventtype == 'discharge')

In [ ]:
transfers.careunit.value_counts(normalize = True)

In [ ]:
transfers.careunit.unique()

In [ ]:
transfers[['intime','outtime']] = pd.to_datetime(transfers[['intime', 'outtime']])

In [ ]:
transfers.info(show_counts = True)

In [ ]:
bool_outs

In [ ]:
transfers.describe()

In [ ]:
bool_discharge = transfers.eventtype=='discharge'
bool_carenan = transfers.careunit.isna()
bool_outtime = transfers.outtime.isna()

In [ ]:
print(all(bool_discharge == bool_carenan))
print(all(bool_discharge == bool_outtime))


In [ ]:
transfers.head()

In [ ]:
services = pd.read_csv('MIMIC IV ED/services.csv')

In [ ]:
services.shape

In [ ]:
services.head()

In [ ]:
services.prev_service.value_counts()

In [ ]:
transfers.eventtype.value_counts()

In [ ]:
transfers.

In [ ]:
emar = pd.read_csv('MIMIC IV/emar.csv')

In [ ]:
emar.medication.nunique()

In [ ]:
emar.head()

In [ ]:
transfers.careunit.value_counts()

In [ ]:
print('Number of patients:',transfers['subject_id'].nunique())
print('Number of transfers (and rows number):',transfers['transfer_id'].nunique())
print('Number of stays:',transfers['hadm_id'].nunique())
print('Number of departments:',transfers['careunit'].nunique())

In [ ]:
sum(transfers['careunit'].isna())

In [ ]:
print('Number of departments:',transfers['careunit'].unique())

In [ ]:
admissions = pd.read_csv('MIMIC IV ED/admissions.csv')

In [ ]:
pd.to_datetime(admissions.admittime,infer_datetime_format = True).describe(datetime_is_numeric = True)

In [ ]:
admissions.discharge_location.unique()

In [ ]:
admissions.head()

In [ ]:
len(admissions)

In [ ]:
print('Number of patients:',admissions['subject_id'].nunique())
print('Number of transfers (and length of rows):',admissions['transfer_id'].nunique())
print('Number of stays:',transfers['hadm_id'].nunique())
print('Number of departments:',transfers['careunit'].nunique())

In [ ]:
edstays['disposition'].value_counts()

In [ ]:
diagnosis.icd_version.value_counts()

In [ ]:
diagnosis['icd_title'].value_counts().head()

In [ ]:
print('\033[1m' + 'diagnosis')
display(diagnosis.head())
display(diagnosis.info())
print('\033[1m' + 'edstays')
display(edstays.head())
display(diagnosis.info())
print('\033[1m' + 'medrecon')
display(medrecon.head())
display(medrecon.info())
print('\033[1m' + 'pyxis')
display(pyxis.head())
display(pyxis.info())
print('\033[1m' + 'triage')
display(triage.head())
display(triage.info())
print('\033[1m' + 'vitalsign')
display(vitalsign.head())
display(vitalsign.info())